In [1]:
from FlexivPy.robot.interface import FlexivDDSClient

In [ ]:
robot =FlexivDDSClient()

In [3]:
from FlexivPy.controllers.taskspace import DiffIKController
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.robot import FlexivCmd
model = FlexivModel()
task_controller = DiffIKController(model, dt=0.01)

In [4]:
import numpy as np
cmd = FlexivCmd()
cmd.q = model.q0
robot.set_cmd(cmd)
s = robot.get_robot_state()
T_cmd = model.getInfo(model.q0, np.zeros(7))['poses']['link7'].copy()
T_cmd[2,-1]+=0.1
q_des = s.q

In [5]:
import time
for i in range(500):
    s = robot.get_robot_state()

    dq_des = task_controller(np.array(s.q), np.array(s.dq), T_cmd).squeeze()
    cmd = FlexivCmd()
    cmd.kp = np.ones(7)*15*0
    cmd.kv = np.ones(7)*8
    cmd.q = q_des
    cmd.dq = dq_des
    robot.set_cmd(cmd)
    time.sleep(task_controller.dt)

## Using The Runners

### Blocking

In [5]:
from FlexivPy.controllers.runners import blocking_runner

In [ ]:
blocking_runner(robot, task_controller)

### Nonblocking

In [13]:
from FlexivPy.controllers.runners import NonBlockingRunner
runner = NonBlockingRunner(robot, task_controller)

In [10]:
runner.close()

In [12]:
info = model.getInfo(model.q0, np.zeros(7))
T_cmd = info['poses']['link7']

In [14]:
task_controller.T_cmd = T_cmd

In [15]:
runner.close()